# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [ ]:
# Ссылка на выбранное вами произведение
# Для примера возьмем Преступление и Наказание
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'

Спарсим текст из файлика.

In [ ]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [ ]:
text = text.lower()

Разобьём весь текст на предложения. 

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [ ]:
sents_tokenize = [tokenizer.tokenize(sent) for sent in sents]
sents_tokenize[4:6]

[['и',
  'каждый',
  'раз',
  'молодой',
  'человек',
  'проходя',
  'мимо',
  'чувствовал',
  'какое',
  'то',
  'болезненное',
  'и',
  'трусливое',
  'ощущение',
  'которого',
  'стыдился',
  'и',
  'от',
  'которого',
  'морщился'],
 ['он',
  'был',
  'должен',
  'кругом',
  'хозяйке',
  'и',
  'боялся',
  'с',
  'нею',
  'встретиться']]

In [ ]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [ ]:
len(words) # всего слов

173403

In [ ]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [ ]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
display(len(stopwords_ru))
stopwords_ru[:10]

151

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [ ]:
sents_tokenize = [[word for word in text_cur if word not in stopwords_ru] for text_cur in sents_tokenize]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=5fd7d2319b16c9a11e708eca72254aee228d6b35305d153d431cadfbc592a0fd
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авеньон и пленил пап!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авеньон и пленить папа'

In [ ]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

Обработаем все слова из датасета. 

In [ ]:
sents_tokenize = [[morph.normal_forms(word)[0] for word in text_cur] for text_cur in sents_tokenize]

In [ ]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [ ]:
len(words) # всего слов

93069

In [ ]:
len(set(words)) # уникальных слов

11084

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хотите больше предобработки, [можете прочитать этот мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [ ]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1449),
 ('всё', 963),
 ('знать', 630),
 ('раскольник', 567),
 ('свой', 549),
 ('один', 548),
 ('сказать', 544),
 ('говорить', 536),
 ('человек', 501),
 ('весь', 442),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('сам', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('петрович', 369)]

In [ ]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4768

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [ ]:
%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(size=100, window=2, min_count=3, workers=4)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

CPU times: user 23.4 s, sys: 291 ms, total: 23.7 s
Wall time: 14.4 s


(7944448, 9306900)

In [ ]:
model.corpus_count # число примеров в обучающей выборке

13565

Смотрим, сколько в модели слов.

In [ ]:
len(model.wv.vocab)

4768

In [ ]:
'старуха' in model.wv.vocab

True

## 3. Свойства модели

In [ ]:
# вектор слова
model.wv['старуха'][:10]

array([-0.8561536 , -1.0795974 , -0.26131397,  0.46300855,  1.6357554 ,
       -0.0353207 ,  0.5920026 ,  1.890924  , -0.78100294, -0.46361428],
      dtype=float32)

In [ ]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [ ]:
# похожести слов 
model.wv.similarity('тварь', 'право')

0.055715106

In [ ]:
model.wv.similarity('старуха', 'топор')

0.4192768

In [ ]:
model.wv.similarity('тварь', 'тварь')

0.99999994

In [ ]:
# самые похожие
model.wv.most_similar('старуха')

[('лизавета', 0.5559487342834473),
 ('обыск', 0.47559523582458496),
 ('близко', 0.4377446472644806),
 ('топор', 0.41927677392959595),
 ('наклепать', 0.41877609491348267),
 ('отметка', 0.3899194002151489),
 ('кулак', 0.38478708267211914),
 ('обратно', 0.3828233480453491),
 ('домой', 0.37311479449272156),
 ('ограбить', 0.3721403479576111)]

In [ ]:
model.wv.most_similar('лестница')

[('ступенька', 0.5415230989456177),
 ('всходить', 0.5337059497833252),
 ('прихожая', 0.5223307609558105),
 ('узенький', 0.4729374647140503),
 ('ступень', 0.4559502601623535),
 ('спускаться', 0.45580971240997314),
 ('коридор', 0.44950059056282043),
 ('настежь', 0.44040998816490173),
 ('дворницкий', 0.43754225969314575),
 ('вниз', 0.4349845349788666)]

In [ ]:
# арифметика
model.most_similar(positive=['раскольников'], negative=['тварь'])[:10]

<ipython-input-45-2a6704991cd2>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['раскольников'], negative=['тварь'])[:10]


[('несколько', 0.3556216359138489),
 ('пристально', 0.2935299575328827),
 ('почтительно', 0.29252225160598755),
 ('прислушиваться', 0.2689328193664551),
 ('незнакомый', 0.26657626032829285),
 ('довольно', 0.2604427933692932),
 ('замешкаться', 0.2600513696670532),
 ('край', 0.2588003873825073),
 ('слушать', 0.253754198551178),
 ('казённый', 0.2519662380218506)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [ ]:
model_path = "./our_w2v.model"
model.save(model_path)

In [ ]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [ ]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [ ]:
text2 = text2.lower()
sents2 = sent_tokenize(text2)

sents_tokenize2 = [tokenizer.tokenize(sent) for sent in sents2]
sents_tokenize2 = [[morph.normal_forms(word)[0] for word in text_cur if word not in stopwords_ru]
                      for text_cur in sents_tokenize2]


In [ ]:
sents_tokenize2[10]

['сени', 'выйти', 'царь', 'отец']

In [ ]:
len(sents_tokenize2)

254

Дополняем модель.

In [ ]:
# обновили словарь
our_model.build_vocab(sents_tokenize2, update=True)

# дообучили
our_model.train(sents_tokenize2, total_examples=our_model.corpus_count, epochs=100)

(177498, 269000)

In [ ]:
'ядро' in model.wv.vocab

False

In [ ]:
'ядро' in our_model.wv.vocab

True

In [ ]:
our_model.wv.most_similar('ядро')

[('изумруд', 0.7330085039138794),
 ('скатерть', 0.6428670883178711),
 ('грызть', 0.6196736097335815),
 ('чистый', 0.6063417196273804),
 ('складка', 0.5761065483093262),
 ('шкатулка', 0.5548331141471863),
 ('серебро', 0.546135425567627),
 ('пристань', 0.5433902740478516),
 ('орешек', 0.5368527770042419),
 ('золотой', 0.5324065089225769)]

Пример со старым словом.

In [ ]:
our_model.wv.most_similar('сын')

[('обеспечить', 0.5876461267471313),
 ('губерния', 0.4981480836868286),
 ('благословить', 0.48843833804130554),
 ('секрет', 0.4776530861854553),
 ('враньё', 0.45808500051498413),
 ('воровать', 0.45804673433303833),
 ('раздражать', 0.45562446117401123),
 ('наущение', 0.4550246000289917),
 ('расспрос', 0.44552212953567505),
 ('приём', 0.4437122344970703)]

In [ ]:
model.wv.most_similar('сын')

[('обеспечить', 0.5480742454528809),
 ('наущение', 0.498038649559021),
 ('секрет', 0.48968130350112915),
 ('преувеличить', 0.4833449721336365),
 ('раздражать', 0.48290401697158813),
 ('приём', 0.46065837144851685),
 ('посредство', 0.45348575711250305),
 ('следить', 0.45188313722610474),
 ('выздоровление', 0.4381413459777832),
 ('родственница', 0.4327961802482605)]